## Joining on Non-Equi Operators

In [28]:
import pandas as pd
import janitor
import numpy as np
from io import StringIO

In [42]:
data = """           A    B       C      D                             E
        0  0     0.0  c    False    1970-01-01 00:00:00.000000768
        1  0    0.0   c    False     1970-01-01 00:00:00.000000768"""

df = pd.read_csv(StringIO(data), sep ='\s{2,}', engine = 'python', parse_dates=['E'])

df

,A,B,C,D,E
0,0,0.0,c,False,1970-01-01 00:00:00.000000768
1,0,0.0,c,False,1970-01-01 00:00:00.000000768


In [43]:
data = """Integers    Numeric  Floats    Strings  Booleans      Dates    Dates_Right
        0         0      0.0     0.0       c      False   1970-01-01 00:00:00.000000000    1970-01-01
        1         -1      0.0     0.0      c       False  1970-01-01 00:01:25.899345921    1970-01-01"""

right = pd.read_csv(StringIO(data), sep ='\s{2,}', engine = 'python', parse_dates=['Dates', 'Dates_Right'])

right

,Integers,Numeric,Floats,Strings,Booleans,Dates,Dates_Right
0,0,0.0,0.0,c,False,1970-01-01 00:00:00.000000000,1970-01-01
1,-1,0.0,0.0,c,False,1970-01-01 00:01:25.899345921,1970-01-01


In [50]:
left_on, right_on = ["E", "Dates"]
df = df.sort_values(left_on, ascending = True)
right = right.sort_values(right_on, ascending = True)
expected = (
    df.assign(t=1)
    .merge(right.assign(t=1), on="t")
    .query(f"{left_on} != {right_on}")
        .groupby(left_on)[right_on].nth(0)
        )

#expected = df.merge(expected, on = left_on, how = 'inner').filter([left_on, right_on])
#expected.index = range(len(expected))

expected


E
1970-01-01 00:00:00.000000768   1970-01-01
Name: Dates, dtype: datetime64[ns]

In [46]:
actual = df.conditional_join(
        right, (left_on, right_on, "!="), how="inner", keep = 'first'
    )
        
actual.filter([left_on, right_on])

,E,Dates
0,1970-01-01 00:00:00.000000768,1970-01-01 00:01:25.899345921
1,1970-01-01 00:00:00.000000768,1970-01-01 00:01:25.899345921


In [47]:
df

,A,B,C,D,E
0,0,0.0,c,False,1970-01-01 00:00:00.000000768
1,0,0.0,c,False,1970-01-01 00:00:00.000000768


In [49]:
right

,Integers,Numeric,Floats,Strings,Booleans,Dates,Dates_Right
0,0,0.0,0.0,c,False,1970-01-01 00:00:00.000000000,1970-01-01
1,-1,0.0,0.0,c,False,1970-01-01 00:01:25.899345921,1970-01-01


In [33]:
#https://stackoverflow.com/q/61948103/7175713 
df1 = pd.DataFrame({'id': [1,1,1,2,2,3], 
                    'value_1': [2,5,7,1,3,4]})

df2 = pd.DataFrame({'id': [1,1,1,1,2,2,2,3], 
                    'value_2A': [0,3,7,12,0,2,3,1], 
                    'value_2B': [1,5,9,15,1,4,6,3]})

In [34]:
df1

,id,value_1
0,1,2
1,1,5
2,1,7
3,2,1
4,2,3
5,3,4


In [35]:
df2

,id,value_2A,value_2B
0,1,0,1
1,1,3,5
2,1,7,9
3,1,12,15
4,2,0,1
5,2,2,4
6,2,3,6
7,3,1,3


Join on equi and non-equi operators is possible:

In [36]:
df1.conditional_join(
        df2,
        ('id', 'id', '=='),
        ('value_1', 'value_2A', '>='),
        ('value_1', 'value_2B', '<='),
        sort_by_appearance = True
    )

left         right                  
    id value_1    id value_2A value_2B
0    1       5     1        3        5
1    1       7     1        7        9
2    2       1     2        0        1
3    2       3     2        2        4
4    2       3     2        3        6

The default join is inner. left and right joins are supported as well:

In [37]:
df1.conditional_join(
        df2,
        ('id', 'id', '=='),
        ('value_1', 'value_2A', '>='),
        ('value_1', 'value_2B', '<='),
        how='left',
        sort_by_appearance = True
    )

left         right                  
    id value_1    id value_2A value_2B
0    1       2   NaN      NaN      NaN
1    1       5   1.0      3.0      5.0
2    1       7   1.0      7.0      9.0
3    2       1   2.0      0.0      1.0
4    2       3   2.0      2.0      4.0
5    2       3   2.0      3.0      6.0
6    3       4   NaN      NaN      NaN

In [38]:
df1.conditional_join(
        df2,
        ('id', 'id', '=='),
        ('value_1', 'value_2A', '>='),
        ('value_1', 'value_2B', '<='),
        how='right',
        sort_by_appearance = True
    )

left         right                  
    id value_1    id value_2A value_2B
0  NaN     NaN     1        0        1
1  1.0     5.0     1        3        5
2  1.0     7.0     1        7        9
3  NaN     NaN     1       12       15
4  2.0     1.0     2        0        1
5  2.0     3.0     2        2        4
6  2.0     3.0     2        3        6
7  NaN     NaN     3        1        3

Join on just the non-equi joins is also possible:

In [39]:
df1.conditional_join(
        df2,
        ('value_1', 'value_2A', '>'),
        ('value_1', 'value_2B', '<'),
        how='inner',
        sort_by_appearance = True
    )

left         right                  
    id value_1    id value_2A value_2B
0    1       2     3        1        3
1    1       5     2        3        6
2    2       3     2        2        4
3    3       4     1        3        5
4    3       4     2        3        6

Join on not equal -> !=

In [40]:
df1.conditional_join(
        df2,
        ('id', 'id', "!=")
    )

left         right                  
    id value_1    id value_2A value_2B
0    1       2     2        0        1
1    1       5     2        0        1
2    1       7     2        0        1
3    2       1     3        1        3
4    2       3     3        1        3
5    3       4     1        0        1

If the columns from both dataframes have nothing in common, a single indexed column is returned:

In [41]:
(df1.select_columns('value_1')
    .conditional_join(
        df2.select_columns('val*'),
        ('value_1', 'value_2A', '>'),
        ('value_1', 'value_2B', '<'),
    )
)

,value_1,value_2A,value_2B
0,2,1,3
1,5,3,6
2,3,2,4
3,4,3,5
4,4,3,6
